<a href="https://colab.research.google.com/github/filosoficaguilar/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods_(Week_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge wget --yes

In [ ]:
import requests
import pandas as pd
import folium # map rendering library
import wget
import matplotlib.pyplot as plt
import numpy as np
import math
from geopy.distance import vincenty
from shapely.geometry import LineString, Point, LinearRing

In [ ]:
df_csv = pd.read_csv(filepath_or_buffer="https://datos.cdmx.gob.mx/explore/dataset/coloniascdmx/download/?format=csv&timezone=America/Mexico_City&lang=es&use_labels_for_header=true&csv_separator=%2C")
df_csv

In [ ]:
df=df_csv.drop(columns=['ENTIDAD','Geo Shape', 'CVE_ALC','ALCALDIA','CVE_COL','SECC_COM','SECC_PAR'],axis=1)
df[['lat','lon']] = df["Geo Point"].str.split(',',expand=True)
df_cdmx = df.drop(columns=["Geo Point"],axis=1)
df_cdmx = df_cdmx.dropna(axis=0)
df_cdmx.reset_index(drop=True, inplace=True)
df_cdmx = df_cdmx.rename(columns={"COLONIA": "Neighborhood"})
df_cdmx

In [ ]:
# create map of Mexico City using latitude and longitude values

latitude = 19.4978
longitude =  -99.1269

map_cdmx = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lon,name in zip( df_cdmx['lat'],df_cdmx['lon'], df_cdmx['Neighborhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx)      
map_cdmx

In [ ]:
CLIENT_ID = "DBDKII3BFJLFQ1H1RLSEQHLC0TC2RMAJ0CJLEETESTRSXARB"
CLIENT_SECRET ="Z2FNS1LXNKSEKZ44ZFWJ0RPWNTRDWJ4PZUMN5UNS3RMPAJRI"
VERSION = "20190425"
GeoId=72057594041458533
place= "Ciudad de México, DF, México"
offset = 100
limit = 100
venues = []

url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&nearGeoId={}&offset={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    place,
    GeoId,
    offset,
    limit)

results = requests.get(url).json()["response"]['groups'][0]['items']
    
for venue in results:
    venues.append((
        name,
        lat, 
        lon, 
        venue['venue']['name'], 
        venue['venue']['location']['lat'], 
        venue['venue']['location']['lng'],  
        venue['venue']['categories'][0]['name']))

In [ ]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Name', 'Latitude','Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df

In [ ]:
# create map of CDMX using latitude and longitude values
#This will show us where are all the venues from Foursquare.com and then we will compare them with all the Neighborhoods
latitude = 19.4978
longitude =  -99.1269

map_venue = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat,lon, name, category in zip(venues_df['VenueLatitude'], venues_df['VenueLongitude'], venues_df['VenueName'], venues_df['VenueCategory']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venue)  
    
map_venue

In [ ]:
latitude = 19.4978
longitude =  -99.1269

map_cdmx_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lon,name in zip( df_cdmx['lat'],df_cdmx['lon'], df_cdmx['Neighborhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx_venues)      

for lat,lon, name, category in zip(venues_df['VenueLatitude'], venues_df['VenueLongitude'], venues_df['VenueName'], venues_df['VenueCategory']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx_venues)  

map_cdmx_venues

<h2>Getting the middle point</h2>

With all the venues and Neighborhoods displayed before we can get the middle point of all the venues to get the best place to stay on a trip

In [ ]:
#Getting the middle point of all the venues
lat_all = []
lon_all = []
for lat,lon in zip(venues_df['VenueLatitude'], venues_df['VenueLongitude']):
  lat_all.append(lat)
  lon_all.append(lon)

lat_final = sum(lat_all)/len(lat_all)
lon_final = sum(lon_all)/len(lon_all)
print(lat_final,lon_final)

In [ ]:
label = '{},\n {},{}'.format("Middle Point of venues",lat_final,lon_final)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [lat_final, lon_final],
    radius=6,
    popup=label,
    color='Green',
    fill=True,
    fill_color='Green',
    fill_opacity=0.7,
    parse_html=False).add_to(map_cdmx_venues)  

map_cdmx_venues

<h2>Getting the distances</h2>

Once we have the middle point we need to know what is the distance between the middle point and the Neighborhoods to make the analysis



In [ ]:
middle_point = (lat_final, lon_final)
neig = df[["lat","lon"]].values
point = Point(middle_point)
route = LineString(neig)
pol_ext = LinearRing(route.coords)
d = pol_ext.project(point)
p = pol_ext.interpolate(d)
closest_point_coords = list(p.coords)[0]

distance = vincenty(closest_point_coords, middle_point).km

print(distance)

In [ ]:
middle_point = (lat_final, lon_final)
neig = df[["lat","lon"]].values
distances = []
for index,i in enumerate(df_cdmx[["lat","lon"]].values):
  x = math.pow(float(i[0])-float(middle_point[0]),2)
  y = math.pow(float(i[1])-float(middle_point[1]),2)
  distances.append(math.sqrt(x+y))

df_cdmx["distance_to_mid"] = distances
df_cdmx = df_cdmx.sort_values(by=['distance_to_mid'])
df_cdmx.reset_index(drop=True)

df_cdmx

<h2>Plotting the distances</h2>

We can see on the next figure that a few Neighborhoods are close enough to the midd point so thats what we wanna know

In [ ]:
ax = df_cdmx.plot(kind='bar',x='Neighborhood',y='distance_to_mid',color='red',figsize=(40,20),fontsize=15)
for i, t in enumerate(ax.get_xticklabels()):
    if (i % 15) != 0:
        t.set_visible(False)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_cdmx

In [ ]:
latitude = 19.4978
longitude =  -99.1269

map_cdmx_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lon,name,distance in zip( df_cdmx['lat'],df_cdmx['lon'], df_cdmx['Neighborhood'],df_cdmx["distance_to_mid"]):
    if(distance<=0.01):
      op = 1
      rad = 6
    elif(distance>0.01 and distance<=0.1):
      op= 0.5
      rad = 3.5
    else:
      op= 0.1
      rad= 2
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=rad,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=op,
        parse_html=False).add_to(map_cdmx_venues)      

for lat,lon, name, category in zip(venues_df['VenueLatitude'], venues_df['VenueLongitude'], venues_df['VenueName'], venues_df['VenueCategory']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='Red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx_venues)  

label = '{},\n {},{}'.format("Middle Point of venues",lat_final,lon_final)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [lat_final, lon_final],
    radius=6,
    popup=label,
    color='Green',
    fill=True,
    fill_color='Green',
    fill_opacity=0.7,
    parse_html=False).add_to(map_cdmx_venues)  


map_cdmx_venues

<h2>Conclusion<h2>

With all the analisys before we can know that the best Neighborhoods to stay on a tri to CDMX are the next



1.   HIPODROMO II
2.   HIPODROMO CONDESA	
3.   ESCANDON I	
4.   CONDESA	
5.   HIPODROMO I

And the worst are 

1.   SAN JUAN TEPENAHUAC (PBLO)
2.   EMILIANO ZAPATA 1A
3. SAN ANDRES MIXQUIC (PBLO)
4. EMILIANO ZAPATA 2A
5. SANTA ANA TLACOTENCO (PBLO)

With these we can plan a very nice holly day's trip
